In [ ]:
!pip -q install datasets
!python -m spacy download en_core_web_lg
from datasets import load_dataset
import spacy
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dataset = load_dataset("paws", "labeled_final")
nlp = spacy.load("en_core_web_lg")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")

dataset = dataset.map(encode, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
def remove_stop(doc):
    filtered_words = [token.text for token in doc if not token.is_stop]
    clean_text = ' '.join(filtered_words)
    return clean_text

In [ ]:
train_data = dataset['test']
tp, tn, fp, fn = 0, 0, 0, 0

for item in train_data:
    sent1 = item['sentence1']
    sent2 = item['sentence2']
    label = item['label']
    doc1 = nlp(sent1)
    doc2 = nlp(sent2)
    nsent1 = remove_stop(doc1)
    nsent2 = remove_stop(doc2)
    doc1 = nlp(nsent1)
    doc2 = nlp(nsent2)
    sim_score = doc1.similarity(doc2)
    # print(sim_score)

    if (sim_score > 0.985):
        if (label == 1):
            tp += 1
        else:
            fp += 1
    else:
        if (label == 1):
            fn += 1
        else:
            tn += 1

print((tp + tn)/(tp + tn + fp + fn))
print(tp, tn, fp, fn)

0.9889391409777297
0.9869503333299119
0.9461691701714584
0.9834821900553993
0.9905733223495539
0.9920853277272469
0.9871780767506172
0.9869504012505028
0.9999999549500723
0.9251363762933481
0.9135444308265139
0.9784897825390132
1.0000000479895197
0.9924977884078985
0.9642529457736427
0.7730238577846965
0.8044750186515268
0.9926495756354582
0.983021177808971
0.9854805347339444
0.922392364111749
0.9833713451303814
0.9807039373639903
1.0
0.9461691717652634
0.9314929942701816
1.0000000348079434
0.9985932258898691
0.9962937629046643
0.720910781328083
0.974905307210106
0.9967186327444328
0.9859876989482689
0.9967215825838144
0.9848249276228906
0.9824340522059402
0.983860236082778
0.9833020365836507
0.981909163263131
1.0
0.9795660125121486
1.0000000113112133
0.9759530201687588
0.9981093618472671
0.993466262129344
0.9928087978896497
1.0000000003904344
0.9950319158127677
0.9999999852217872
0.9544615456580957
0.996409192788015
0.9869458634393211
0.9869932213465569
0.9540017586397628
0.9643530803